# Intro to Data Science
## Part IV. - Dimensionality Reduction

### Table of contents

- ##### Dimensionality reduction:
    - <a href="#What-is-Dimensionality-Reduction?">Theory</a>
    - <a href="#1.-Feature-Selection">Feature Selection</a>
    - <a href="#2.-Matrix-Decomposition">Matrix Decomposition</a>

- ##### SVM
    - <a href="#SVM-=-Support-Vector-Machines">Theory</a>
    - <a href="#Example">Example</a>

- ##### Feature Union
    - <a href="#Feature-Unions">Feature Union</a>
    - <a href="#Create-custom-transformers">Custom transformers</a>
    - <a href="#Exercise:-Prediction-on-last-week's-dataset">Exercise</a>
    
---

## What is Dimensionality Reduction?
Dimensionality reduction _"is the process of reducing the number of random variables under consideration, and can be divided into feature selection and feature extraction."_

_"__Feature selection__ approaches try to find a subset of the original variables. ... In some cases, data analysis such as regression or classification can be done in the reduced space more accurately than in the original space."_

_"__Feature extraction__ transforms the data in the high-dimensional space to a space of fewer dimensions. The data transformation may be linear, as in principal component analysis (PCA), but many nonlinear dimensionality reduction techniques also exist."_ from: <a href="https://en.wikipedia.org/wiki/Dimensionality_reduction">Wiki</a>


## Why is it important?
With hundreds of features in the datasets, there will always be some which does not contribute to the overall precision of the predictive model. These features could be redundant, overlapping or linear combination of each other or simply irrelevant to the prediction. To improve training and transformation/prediction time, it is crucial to reduce the number of features to a moderate amount.

### <a href="https://en.wikipedia.org/wiki/Curse_of_dimensionality">The curse of dimensionality</a>

<img src="http://tm.durusau.net/wp-content/uploads/2016/06/curse-dimensions-460.png" align="left"> 
<br style="clear:left;"/>
(<a href="http://tm.durusau.net/wp-content/uploads/2016/06/curse-dimensions-460.png">source</a>)

This is our greatest enemy, after over/underfitting. With the increase of dimensions, the number of possible states or input vectors grow **exponentially**. Even in the most basic case of binary variables, in a moderate amount of say, 50 dimensions, we'll have $2^{50} > 10^{15}$ number of possible inputs. This means that for the same effectiveness, **we need exponentially more training points**!

## Tools
- <a href="http://scikit-learn.org/stable/modules/feature_selection.html">Feature Selection</a>
- <a href="http://scikit-learn.org/stable/modules/decomposition.html#decompositions">Matrix decomposition</a>
- <a href="http://scikit-learn.org/stable/modules/feature_extraction.html#feature-hashing">Hashing</a>
- etc.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d, Axes3D
from matplotlib.colors import ListedColormap
import seaborn as sns

import numpy as np
import scipy.sparse as sp
import pandas as pd

from sklearn.datasets import load_iris

from sklearn.pipeline import Pipeline

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression

In [ ]:
def plotme(X, y):
    with sns.color_palette('muted', n_colors=3) as mycolors:
        plt.scatter(*X.T, c=y, cmap=ListedColormap(mycolors), edgecolors='k')

def plot_results_with_hyperplane(clf, clf_name, df, ax):
    x_min, x_max = df.x.min() - .5, df.x.max() + .5
    y_min, y_max = df.y.min() - .5, df.y.max() + .5

    # step between points. i.e. [0, 0.02, 0.04, ...]
    step = .02
    # to plot the boundary, we're going to create a matrix of every possible point
    # then label each point using our classifier
    xx, yy = np.meshgrid(np.arange(x_min, x_max, step), np.arange(y_min, y_max, step))
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    # this gets our predictions back into a matrix
    Z = Z.reshape(xx.shape)
    
    # plot the boundaries
    ax.pcolormesh(xx, yy, Z, cmap=plt.cm.Paired)
    ax.scatter(xs, ys, c=['r' if c else 'b' for c in cs], edgecolors='k')
    ax.set_title(clf_name)

In [ ]:
iris = load_iris()
X, y = iris.data, iris.target

## 1. Feature Selection

### Simple (variance) threshold based:

_"VarianceThreshold is a simple baseline approach to feature selection. It removes all features whose variance doesn’t meet some threshold. By default, it removes all zero-variance features, i.e. features that have the same value in all samples."_ from: <a href="http://scikit-learn.org/stable/modules/feature_selection.html#removing-features-with-low-variance">sklearn docs</a>

In [ ]:
from sklearn.feature_selection import VarianceThreshold

thres = VarianceThreshold(.6)
X_t = thres.fit_transform(X)
X_t.shape, zip(iris.feature_names, thres.variances_)

In [ ]:
plotme(X_t, y)

### Recursive Feature Elimination (RFE):

_"Given an external estimator that assigns weights to features (e.g., the coefficients of a linear model), __recursive feature elimination (RFE)__ is to select features by recursively considering smaller and smaller sets of features. First, the estimator is trained on the initial set of features and weights are assigned to each one of them. Then, features whose absolute weights are the smallest are pruned from the current set features. That procedure is recursively repeated on the pruned set until the desired number of features to select is eventually reached."_ from: <a href="http://scikit-learn.org/stable/modules/feature_selection.html#recursive-feature-elimination">sklearn docs</a>

In [ ]:
from sklearn.feature_selection import RFE

rfe = RFE(LinearRegression(), n_features_to_select=2)
X_t = rfe.fit_transform(X, y)
X_t.shape, zip(iris.feature_names, rfe.ranking_)

In [ ]:
plotme(X_t, y)

Thought experiment: Consider the __<a href="http://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_digits.html">digits</a>__ dataset and try to describe the results found __<a href="http://scikit-learn.org/stable/auto_examples/feature_selection/plot_rfe_digits.html#recursive-feature-elimination">here</a>__.


### Select based on models:

This method is very versatile since it is based on an external model. The features are selected based on the external model's coefficients. If a feature is under a set threshold, it is considered unimportant and removed.  
In sklearn, the `SelectFromModel` transformator requires an estimator that has a `coef_` or `feature_importances_` attribute.

In [ ]:
from sklearn.feature_selection import SelectFromModel

sel = SelectFromModel(LogisticRegression(C=.1))
X_t = sel.fit_transform(X, y)
X_t.shape, zip(iris.feature_names, sel.get_support())

In [ ]:
plotme(X_t, y)

## 2. Matrix Decomposition

### Principal Component Analysis (PCA):

_"The main linear technique for dimensionality reduction, principal component analysis, performs a linear mapping of the data to a lower-dimensional space in such a way that the variance of the data in the low-dimensional representation is maximized. In practice, the covariance (and sometimes the correlation) matrix of the data is constructed and the eigen vectors on this matrix are computed. The eigen vectors that correspond to the largest eigenvalues (the principal components) can now be used to reconstruct a large fraction of the variance of the original data. Moreover, the first few eigen vectors can often be interpreted in terms of the large-scale physical behavior of the system. The original space (with dimension of the number of points) has been reduced (with data loss, but hopefully retaining the most important variance) to the space spanned by a few eigenvectors."_ from: <a href="https://en.wikipedia.org/wiki/Dimensionality_reduction#Principal_component_analysis_.28PCA.29">Wiki</a>


In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
X_t = pca.fit_transform(X)
X_t.shape, pca.explained_variance_ratio_

In [ ]:
plotme(X_t, y)

A notebook with the results shown in 3d space can be downloaded from <a href="http://scikit-learn.org/stable/_downloads/plot_pca_iris.ipynb">here</a>.


### Singular Value Decomposition (SVD):

A well known matrix factorization method which has been widely used in many field of statistics, signal processing, etc. It is basicly a matrix decomposition method which from an $m{\times}n$ matrix $M$ creates 3 matrices. 

$$M=U{\Sigma}V^T$$

where:

- $U$ - $m{\times}m$ (orthogonal matrix)
- $\Sigma$ - $m{\times}n$ (diagonal matrix)
- $V$ - $n{\times}n$ (orthogonal matrix)

The ${\Sigma}$ matrix's diagonal contains $M$ matrix's singular values (square roots of eigenvalues) in descending order. In the case of feature extraction, only the first _k_ dimensions are left.
$$X \approx X_k = U_k \Sigma_k V_k^\top$$

In [ ]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=2)
X_t = svd.fit_transform(X)
X_t.shape, svd.explained_variance_ratio_

In [ ]:
plotme(X_t, y)

---

## Model of the week:
## SVM = Support Vector Machines


Behold, the first truly **black box** classifier (note: SVM-s can also be used for regression). Don't bother with the weird name, it will make sense later.  
**Why are SVMs awesome?** Because it is a nonlinear classifier *while it is a linear classifier*. What? Yes! SVM-s do linear classification on the data **after it is transformed**.  
  
Let's look at the linear 2D case. There can be many solutions to the problem "Find the separating hyperplane", as seen in the picture below:
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/2/20/Svm_separating_hyperplanes.png/251px-Svm_separating_hyperplanes.png">  
However, they are obviously not equally good. $H_3$ doesn't separate the points according to the class, while $H_1$ and $H_2$ do. But if we look at $H_1$ and $H_2$, **we "feel" that $H_2$ is a better separator**, because there is less ambiguity in classifying the train points (look at the points closest to $H_1$ - they aren't far from being in the other class!).  
The points closest to the possible boundaries are called **support vectors**. **SVM-s try to maximize the margins around the separator**, only points close to the decision boundary affect optimality. The boundary would change if we remove one of the support vectors.  
  

### Example
Let's take a look at an example (plotting function is from <a href="http://blog.yhat.com/posts/why-support-vector-machine.html">here</a>):

In [ ]:
from sklearn.datasets import make_circles

from sklearn import linear_model
from sklearn import tree
from sklearn import svm

We generate 500 points, and classify them according to an imaginary circle:

In [ ]:
xs = np.random.rand(500)*5
ys = np.random.rand(500)*5
cs = np.int0((xs-3)**2 + (ys-2)**2 > 3)

df = pd.DataFrame(data={'x': xs, 'y': ys, 'c': cs})
train_cols = ['x', 'y']

In [ ]:
clfs = {
    "SVM": svm.SVC(),
    "Logistic" : linear_model.LogisticRegression(),
    "Tree": tree.DecisionTreeClassifier()
}

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(15,5))

for i, (clf_name, clf) in enumerate(clfs.items()):
    clf.fit(df[train_cols], df.c)
    plot_results_with_hyperplane(clf, clf_name, df, ax[i])

#### How the heck is this linear?

It is linear in the *transformed space*. If we introduce a third dimension, which we get like this:

In [ ]:
zs = (xs-3)**2 + (ys-2)**2

Then our data points will look like this in the 3D space:

In [ ]:
fig = plt.figure()
ax = fig.gca(projection='3d')
ax.scatter3D(xs, ys, zs, c=cs, cmap=plt.cm.Paired)
ax.view_init(10, 30)

Now we can see that the data points can be separated by a plane in this 3D space. Then projecting the intersection of the plane and the function $(x_1-3)^2 + (x_2-2)^2$ back to 2D, we get the classification boundary

In [ ]:
fig = plt.figure()
ax = fig.gca()
ax.scatter(xs, ys, c=cs, cmap=plt.cm.Paired, edgecolors='k')
ax.add_patch(plt.Circle((3,2), radius=np.sqrt(3), fill=False, linewidth=.7))
fig.set_figwidth(4)
fig.set_figheight(4)

There are videos that make this waaay more clear: <a href="https://www.youtube.com/watch?v=9NrALgHFwTo">vid1</a>, <a href="https://www.youtube.com/watch?v=3liCbRZPrZA">vid2</a>

$\renewcommand{\vec}[1]{\mathbf{#1}}$
### Kernel trick
  
But calculating these additional dimensions can be computationally heavy. Fortunately the algorithms only need the dot product of the transformed vectors, not the transformed vectors themselves! This way we only need a **kernel function** that tells us what the product of two transformed vectors are: $ K(\vec{x},\vec{y}) = \phi(\vec{x}) \phi(\vec{y})$.  
  
For example if we specify a simple polynomial kernel function in two dimensions $\phi(\vec{x})\phi(\vec{y}) = K(\vec{x},\vec{y}) = (1+\vec{xy})^2$, where $\vec{x} = (x_1, x_2)$ and $\vec{y} = (y_1, y_2)$, we don't see immediately what $\phi$ transformation corresponds to this. Let's see! 
$$K(\vec{x},\vec{y}) = (1+\vec{xy})^2 = (1+x_1y_1+x_2y_2)^2 = (1 + x_1^2y_1^2 + x_2^2y_2^2 + 2x_1y_1 + 2x_2y_2 + 2x_1x_2y_1y_2)$$
With a bit of thinking, we can see that this is a dot product of the 6 dimensional vectors 
$$\vec{x'} = \phi(\vec{x}) = (1,x_1^2,x_2^2,\sqrt{2}x_1,\sqrt{2}x_2,\sqrt{2}x_1x_2)$$ and 
$$\vec{y'} = \phi(\vec{y}) = (1,y_1^2,y_2^2,\sqrt{2}y_1,\sqrt{2}y_2,\sqrt{2}y_1y_2)$$! So the transformation function is $\phi(\vec{x}) = \phi(x_1, x_2) = \vec{x'} = (1,x_1^2,x_2^2,\sqrt{2}x_1,\sqrt{2}x_2,\sqrt{2}x_1x_2)$.  
  
The point of kernel functions is that we don't *need* the transformations themselves, only the dot products! This example just shows that kernel functions aren't some form of black magic.
  
Take a look at some <a href="http://scikit-learn.org/stable/modules/svm.html#svm-kernels">common</a> kernels that people use.

---

## Feature Unions

Feature unions are "parallel pipes". Every transformator in the union is applied to the input data, and the results are concatenated. It is very useful if we want to create new features from appling different transformers on the same data.

Not only the transformators steps can be set, but also weight can be associated with them.

In [ ]:
from sklearn.pipeline import FeatureUnion

In [ ]:
feat = FeatureUnion(transformer_list=[
    ('thres', VarianceThreshold(.7)),
    ('svd', TruncatedSVD(n_components=2))
])

FeatureUnion can be a step in a pipeline:

In [ ]:
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import accuracy_score

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
pipe = Pipeline([
    ('norm', StandardScaler()),
    ('feat', feat),
    ('knn', KNeighborsClassifier())
])

In [ ]:
pipe.fit(X_train, y_train)
accuracy_score(y_test, pipe.predict(X_test))

Pipes can be part of unions:

In [ ]:
union = FeatureUnion([
    ('normsvd', Pipeline([('norm', StandardScaler()),
                          ('svd', TruncatedSVD(n_components=2))])),
    ('pca', PCA('mle'))
])

And put this into a pipeline:

In [ ]:
pipe = Pipeline([
    ('feat', union),
    ('knn', KNeighborsClassifier())
])

In [ ]:
pipe.fit(X_train, y_train)
accuracy_score(y_test, pipe.predict(X_test))

# PIPECEPTION!

---
 
### Create custom transformers

Sometimes we just couldn't find what we are looking for in sklearn's massive library. In this case we can write our own transformers.  
It's pretty easy:

- Import the baseclasses

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

- Subclass our transformer

In [ ]:
class Multiplier(BaseEstimator, TransformerMixin):
    
    def __init__(self, multitude):
        self.multitude = multitude

    def fit(self, x, y=None):
        return self

    def transform(self, X, y=None):
        return X * self.multitude

- We are good to go!

In [ ]:
np.arange(1, 21).reshape(4, 5)

In [ ]:
multi = Multiplier(5)
multi.transform(np.arange(1, 21).reshape(4, 5))

In [ ]:
multi.transform(X_test)[:10]

---

## Exercise: Prediction on last week's dataset

- Use last week's dataset
- Transform the nominal features
- Transform the numerical features
- Use the custom transformer from the cheat sheet
- Create a feature union from the nominal and the numerical feature pipes
- Create a pipe with the feature union and a model of your liking
- Predict!